# 술 추천 알고리즘

도수 유사도는 `euclidean distance`로 구했고, 풍미 및 기타 정보의 유사도는 `cosine similarity`로 구했다.  
술의 고유 id를 받아 비슷한 술의 고유 id를 return한다.

### general_recommendation():
도수 유사도 50% 풍미(6가지) 유사도 50% 고려해 주종 상관 없이 비슷한 술을 찾아줌

### wine_recommendation():
도수 유사도 50% 기타(category, origin, producer, wine_grape, flavour) 50% 고려해 같은 대분류 내에서 비슷한 와인을 찾아줌

### the used variables of each alcohols
(현재는 wine recommendation만 구축됨! makgeoli, beer, vodka, soju, whisky, korean도 variable만 바꾸어주어 만들면 됨.)  

all: percent, flavour  
wine: category, percent, **origin**, **producer**, **wine_grape**, flavour  
makgeoli: category, percent, **producer**, flavour  
beer: category, percent, **origin**, **producer**, flavour  
vodka: **origin**, producer, flavour  
soju: category, percent, **producer**, flavour  
whisky: category, **whisky_category**, percent, **origin**, flavour  
korean: category, percent, flavour  

### the explanation of variables
id: 술의 고유 id  
class: 주종  
category: 중분류  
name: 이름  
percent: 도수  
origin: 원산지/생산지  
producer: 제조사/생산자  
wine_grape: 포도 품종 (와인만)  
whisky_category: 위스키 소분류 (위스키만)  
sweet: 달콤한  
light: 가벼운  
soft: 부드러운  
bitter: 쓴맛이 강한  
clean: 깔끔한  
smell: 향이 강한  

In [1]:
# 여러 주종 내 추천
# 주종에 상관 없이 도수와 풍미만 고려해 비슷한 술을 추천해줌
# 항목: 도수, 풍미 (약 6개 항목)

import numpy as np
import pandas as pd 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial import distance_matrix
from scipy.spatial.distance import squareform
from sklearn.preprocessing import MinMaxScaler

data = pd.read_csv("data/drinksData.csv")

# 확인
data.head()

,id,class,category,name,percent,origin,producer,wine_grape,whisky_category,sweet,light,soft,bitter,clean,smell
0,1,wine,와인-레드,가,13,프랑스,가,피노누아,NaN,1,1,0,1,1,0
1,2,wine,와인-스파클링,나,11,칠레,갸,네비올로,NaN,0,0,0,1,1,1
2,3,wine,와인-레드,다,14,이탈리아,거,가르나차,NaN,1,0,0,1,1,1
3,4,wine,와인-스파클링,라,15,스페인,겨,피노누아,NaN,0,0,1,1,1,0
4,5,wine,와인-레드,마,12,프랑스,고,네비올로,NaN,0,1,1,1,0,0


In [2]:
# 도수 유사도 구하기
percent = data['percent']
dist_pair = []

# y축에 임의로 0을 부여한 거리 순서쌍 생성
for i in range(0,len(percent)):
    temp = []
    temp.append(data.loc[i]['percent'])
    temp.append(0)
    dist_pair.append(temp)

# get a distance matrix
df = pd.DataFrame(dist_pair, columns=['x', 'y'])
dist_matrix = distance_matrix(df.values, df.values)

# 정규화
min_max_scaler = MinMaxScaler()
regularised = min_max_scaler.fit_transform(dist_matrix)

# 1에서 빼줘서 더 가까운 것이 우선순위를 갖도록 변경하기
# 데이터 숫자만큼으로 변경해줘야 함
one_matrix = np.ones((22,22))

final_dist = one_matrix - regularised

# 확인
print(final_dist)

[[1.         0.5        0.8        0.66666667 0.66666667 0.4
  0.33333333 1.         0.5        0.8        0.66666667 0.66666667
  0.4        0.33333333 1.         0.5        0.8        0.66666667
  0.66666667 0.4        0.33333333 1.        ]
 [0.5        1.         0.4        0.33333333 0.66666667 0.8
  0.66666667 0.5        1.         0.4        0.33333333 0.66666667
  0.8        0.66666667 0.5        1.         0.4        0.33333333
  0.66666667 0.8        0.66666667 0.5       ]
 [0.75       0.25       1.         0.83333333 0.33333333 0.2
  0.16666667 0.75       0.25       1.         0.83333333 0.33333333
  0.2        0.16666667 0.75       0.25       1.         0.83333333
  0.33333333 0.2        0.16666667 0.75      ]
 [0.5        0.         0.8        1.         0.         0.
  0.         0.5        0.         0.8        1.         0.
  0.         0.         0.5        0.         0.8        1.
  0.         0.         0.         0.5       ]
 [0.75       0.75       0.6        0.5   

In [3]:
# flavour word list 만들기
flavour_list = [] # empty list

for i in range(0, len(data)):
    temp = ""
    
    if data.loc[i]['sweet'] == 1:
        temp = temp + "sweet "
    if data.loc[i]['light'] == 1:
        temp = temp + "light "
    if data.loc[i]['soft'] == 1:
        temp = temp + "soft "
    if data.loc[i]['bitter'] == 1:
        temp = temp + "bitter "
    if data.loc[i]['clean'] == 1:
        temp = temp + "clean "
    if data.loc[i]['smell'] == 1:
        temp = temp + "smell "
    
    flavour_list.append(temp)

# 해당 리스트 데이터 프레임에 추가
data["flavour"] = pd.DataFrame({"flavour":flavour_list})
flavour = data['flavour']

# flavour 코사인 유사도 구하기
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(flavour)

# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)

# 확인
print(cosine_sim)

[[1.         0.57735027 0.75       0.57735027 0.57735027 0.67082039
  0.5        0.67082039 0.         0.70710678 0.28867513 0.5
  0.57735027 0.         0.8660254  0.57735027 0.57735027 0.75
  0.         0.8660254  0.57735027 0.57735027]
 [0.57735027 1.         0.8660254  0.66666667 0.33333333 0.51639778
  0.57735027 0.51639778 0.40824829 0.40824829 0.66666667 0.57735027
  0.33333333 0.         0.33333333 0.66666667 0.         0.8660254
  0.         0.33333333 0.66666667 0.66666667]
 [0.75       0.8660254  1.         0.57735027 0.28867513 0.67082039
  0.5        0.67082039 0.35355339 0.35355339 0.57735027 0.5
  0.57735027 0.         0.57735027 0.8660254  0.28867513 1.
  0.         0.57735027 0.57735027 0.57735027]
 [0.57735027 0.66666667 0.57735027 1.         0.66666667 0.51639778
  0.57735027 0.51639778 0.40824829 0.40824829 0.66666667 0.57735027
  0.66666667 0.         0.33333333 0.33333333 0.33333333 0.57735027
  0.57735027 0.33333333 0.33333333 0.33333333]
 [0.57735027 0.33333333 0

In [4]:
# 도수, flavour를 모두 고려한 similarity 구하기 (weight는 각각 0.5)
new_sim = 0.5 * cosine_sim + 0.5 * final_dist

print(new_sim)

[[1.         0.53867513 0.775      0.62200847 0.62200847 0.5354102
  0.41666667 0.8354102  0.25       0.75355339 0.4776709  0.58333333
  0.48867513 0.16666667 0.9330127  0.53867513 0.68867513 0.70833333
  0.33333333 0.6330127  0.4553418  0.78867513]
 [0.53867513 1.         0.6330127  0.5        0.5        0.65819889
  0.62200847 0.50819889 0.70412415 0.40412415 0.5        0.62200847
  0.56666667 0.33333333 0.41666667 0.83333333 0.2        0.59967937
  0.33333333 0.56666667 0.66666667 0.58333333]
 [0.75       0.5580127  1.         0.7053418  0.31100423 0.4354102
  0.33333333 0.7104102  0.3017767  0.6767767  0.7053418  0.41666667
  0.38867513 0.08333333 0.66367513 0.5580127  0.64433757 0.91666667
  0.16666667 0.38867513 0.37200847 0.66367513]
 [0.53867513 0.33333333 0.68867513 1.         0.33333333 0.25819889
  0.28867513 0.50819889 0.20412415 0.60412415 0.83333333 0.28867513
  0.33333333 0.         0.41666667 0.16666667 0.56666667 0.78867513
  0.28867513 0.16666667 0.16666667 0.41666667

In [5]:
# 여러 주종 내 추천
# 주종에 상관 없이 도수와 풍미만 고려해 비슷한 술을 추천해줌
# 항목: 도수, 풍미 (약 6개 항목)

# 고유 id를 넣으면 해당 술과 비슷한 Top 10의 id를 return
def general_recommendation(input_id, new_sim = new_sim):
    
    # 고유 id로 index 찾기 
    idx = data.index[data['id'] == input_id].tolist() # Int64Index 형식이라 list로 바꾸어줌
    
    # 해당 index의 유사도 리스트 sort in descending order
    score_series = pd.Series(new_sim[idx[0]]).sort_values(ascending = False)
    
    # 유사도 Top 10의 index 추출
    top_10_indexes = list(score_series.iloc[1:11].index)

    # 유사도 1인 항목이 하나 더 있어서 자기 자신이 포함되는 경우에는 자신을 뺀 Top 10의 index 재추출
    if top_10_indexes[0] == idx[0]:
        top_10_indexes = list(score_series.iloc[1:12].index)
        top_10_indexes.remove(idx[0])
    
    # 고유 id를 담기 위한 empty list 생성
    top_10_id = []
    
    # id list
    for i in top_10_indexes:
        id = data.loc[i]['id']
        top_10_id.append(id)
    
    return top_10_id

# test
print(general_recommendation(10))

[22, 1, 17, 3, 4, 18, 15, 5, 8, 12]


In [6]:
# wine word list 만들기
wine_list = [] # empty list

# wine index list 추출
wine_idx = data.index[data['class'] == "wine"].tolist()

# wine: category, percent, origin, producer, wine_grape, flavour
for i in wine_idx:
    temp = ""
    temp = temp + data.loc[i]['category'] + " " + data.loc[i]['origin'] + " " + data.loc[i]['producer'].replace(" ", "") + " " + data.loc[i]['wine_grape'].replace(" ", "") + " "
    
    for flavour in data.loc[i]['flavour']:
        temp = temp + flavour

    wine_list.append(temp)

# wine 코사인 유사도 구하기
# instantiating and generating the count matrix
count = CountVectorizer()
wine_matrix = count.fit_transform(wine_list)

# generating the cosine similarity matrix
wine_sim = cosine_similarity(wine_matrix, wine_matrix)

# wine 도수 유사도 구하기
wine_pair = []

# y축을 임의로 0을 부여한 거리 순서쌍 생성
for i in wine_idx:
    temp = []
    temp.append(data.loc[i]['percent'])
    temp.append(0)
    wine_pair.append(temp)

# get a distance matrix
wine_df = pd.DataFrame(wine_pair, columns=['x', 'y'])
wine_matrix = distance_matrix(wine_df.values, wine_df.values)

# 정규화
min_max_scaler = MinMaxScaler()
wine_regularised = min_max_scaler.fit_transform(wine_matrix)

# 1에서 빼줘서 더 가까운 것이 우선순위를 갖도록 변경하기
wine_one_matrix = np.ones((len(wine_idx),len(wine_idx)))

wine_final_dist = wine_one_matrix - wine_regularised

# 도수 유사도, 타 정보 유사도 각각 0.5씩 weight 부여 후 새로운 matrix 생성
wine_new_sim = 0.5 * wine_sim + 0.5 * wine_final_dist

# 확인
print(wine_new_sim)

[[1.         0.45044593 0.7125     0.60059458 0.66740989 0.43570226
  0.47916667 0.73570226 0.46650635 0.76084392 0.46696395 0.57050416
  0.53407655 0.34344336 0.76726124 0.58407655 0.73407655 0.58333333
  0.57050416 0.46726124 0.50074322 0.76726124]
 [0.45044593 1.         0.46726124 0.45238095 0.54761905 0.71497039
  0.53377926 0.56497039 0.65430335 0.43145502 0.52380952 0.50236418
  0.61428571 0.61680669 0.46428571 0.71428571 0.34285714 0.56755853
  0.41784876 0.68571429 0.54761905 0.60714286]
 [0.6875     0.39226124 1.         0.6171126  0.3671126  0.39462783
  0.39583333 0.61070226 0.41367513 0.71650635 0.68392791 0.48289443
  0.30044593 0.26011003 0.70907655 0.45907655 0.70044593 0.79166667
  0.40383749 0.30044593 0.41740989 0.57544593]
 [0.51726124 0.28571429 0.60044593 1.         0.21428571 0.25197632
  0.26726124 0.56497039 0.15430335 0.63145502 0.78571429 0.25354628
  0.35714286 0.09449112 0.46428571 0.28571429 0.54285714 0.76726124
  0.25354628 0.28571429 0.14285714 0.535714

In [7]:
# 해당 id의 술이 wine인지 체크
def is_wine(input_id):
    temp_idx = data.index[data['id'] == input_id].tolist() # Int64Index 형식이라 list로 바꾸어줌
    result = data.loc[temp_idx[0]]['class'] == "wine"
    return result

# 와인의 고유 id를 넣으면 해당 와인과 비슷한 Top 10 와인의 id를 return
def wine_recommendation(input_id, wine_new_sim = wine_new_sim):
    
    # 고유 id로 index 찾기 
    idx = data.index[data['id'] == input_id].tolist() # Int64Index 형식이라 list로 바꾸어줌
    
    # wine_idx list 내에서 몇번째 와인인지 구하기
    w_idx = wine_idx.index(idx[0])
    
    # 해당 index의 유사도 리스트 sort in descending order
    score_series = pd.Series(wine_new_sim[w_idx]).sort_values(ascending = False)
    
    # 유사도 Top 10의 index 추출
    wine_top_10_indexes = list(score_series.iloc[1:11].index)

    # 유사도 1인 항목이 하나 더 있어서 자기 자신이 포함되는 경우에는 자신을 뺀 Top 10의 index 재추출
    if wine_top_10_indexes[0] == w_idx:
        wine_top_10_indexes = list(score_series.iloc[1:12].index)
        wine_top_10_indexes.remove(w_idx)
    
    # 고유 id를 담기 위한 empty list 생성
    wine_top_10_id = []
    
    # id list
    for i in wine_top_10_indexes:
        index = wine_idx[i] # wine list에서 몇번째인지가 아니라 전체 술 list에서 몇번째인지 구함
        id = data.loc[index]['id']
        wine_top_10_id.append(id)
    
    return wine_top_10_id

In [8]:
# test
# 테스트로 임의의 아이디 넣음
test_num = 2

result = (general_recommendation(test_num))
print(result)

print(is_wine(test_num))

print(wine_recommendation(test_num))

[16, 9, 21, 6, 3, 7, 12, 18, 22, 20]
True
[6, 16, 20, 9, 14, 13, 22, 18, 8, 5]


In [9]:
type(result)

list

In [10]:
df = data.loc[data['id'].isin(result)]
df.head()

,id,class,category,name,percent,origin,producer,wine_grape,whisky_category,sweet,light,soft,bitter,clean,smell,flavour
2,3,wine,와인-레드,다,14,이탈리아,거,가르나차,NaN,1,0,0,1,1,1,sweet bitter clean smell
5,6,wine,와인-스파클링,바,10,칠레,교,가르나차,NaN,1,1,1,0,1,1,sweet light soft clean smell
6,7,wine,와인-레드,사,9,이탈리아,구,피노누아,NaN,0,1,1,0,1,1,light soft clean smell
8,9,wine,와인-레드,자,11,프랑스,그,가르나차,NaN,0,0,1,0,0,1,soft smell
11,12,wine,와인-레드,타,12,스페인,갸,가르나차,NaN,0,0,0,1,0,0,bitter


In [11]:
js = df.to_json(orient='columns')
print(js)

{"id":{"2":3,"5":6,"6":7,"8":9,"11":12,"15":16,"17":18,"19":20,"20":21,"21":22},"class":{"2":"wine","5":"wine","6":"wine","8":"wine","11":"wine","15":"wine","17":"wine","19":"wine","20":"wine","21":"wine"},"category":{"2":"\uc640\uc778-\ub808\ub4dc","5":"\uc640\uc778-\uc2a4\ud30c\ud074\ub9c1","6":"\uc640\uc778-\ub808\ub4dc","8":"\uc640\uc778-\ub808\ub4dc","11":"\uc640\uc778-\ub808\ub4dc","15":"\uc640\uc778-\ub808\ub4dc","17":"\uc640\uc778-\uc2a4\ud30c\ud074\ub9c1","19":"\uc640\uc778-\uc2a4\ud30c\ud074\ub9c1","20":"\uc640\uc778-\ub808\ub4dc","21":"\uc640\uc778-\uc2a4\ud30c\ud074\ub9c1"},"name":{"2":"\ub2e4","5":"\ubc14","6":"\uc0ac","8":"\uc790","11":"\ud0c0","15":"\ub108","17":"\ub7ec","19":"\ubc84","20":"\uc11c","21":"\uc5b4"},"percent":{"2":14,"5":10,"6":9,"8":11,"11":12,"15":11,"17":15,"19":10,"20":9,"21":13},"origin":{"2":"\uc774\ud0c8\ub9ac\uc544","5":"\uce60\ub808","6":"\uc774\ud0c8\ub9ac\uc544","8":"\ud504\ub791\uc2a4","11":"\uc2a4\ud398\uc778","15":"\uc2a4\ud398\uc778","17":"\u

In [ ]:
from flask import Flask
from flask import jsonify

app = Flask(__name__)

@app.route("/")
def test() :
    return js

if __name__ == "__main__" :
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Oct/2022 19:02:05] "GET / HTTP/1.1" 200 -
